In [4]:
import pandas as pd
import numpy as np
import sys
import os
import re
import datetime 
import networkx
import json
from konlpy.tag import Komoran, Kkma, Twitter
from kss import split_sentences
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
def tokenize(content):
    #split sentence
    if(type(content) == str):
        content = content.split('\n')

    sent_list = [sent for sent in split_sentences(''.join(content)) if sent is not None]
    return sent_list

In [22]:
class TextRank:
    def __init__(self):
        self.posTagger = Twitter()
        self.graph2 = []
        self.nouns = []
        self.sentences = []
 
    def loadSentence(self, sentenceToken):    
        self.sentences = sentenceToken   
        for sentence in self.sentences:
            if sentence is not '':
                self.nouns.append(' '.join([noun for noun in self.posTagger.nouns(str(sentence)) 
                                       if len(noun) > 1]))
    
    def get_rank(self, d=0.85):
        matrix_size = self.graph2.shape[0]
        for id in range(matrix_size):
            self.graph2[id, id] = 0
            link_sum = np.sum(self.graph2[:,id])
            if link_sum != 0:
                self.graph2[:, id] /= link_sum
            self.graph2[:, id] *= -d
            self.graph2[id, id] = 1
        graph_mat = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(self.graph2, graph_mat)
        return {idx: r[0] for idx, r in enumerate(ranks)}
    
    def build(self, sentence):
        tfidf = TfidfVectorizer()
        count = CountVectorizer()
        tfidf_mat = tfidf.fit_transform(sentence).toarray()
        self.graph2 = np.dot(tfidf_mat, tfidf_mat.T)
 

    def summarize(self, ratio):
        summary = []
        index=[]

        idx_rank = self.get_rank()
        sorted_rank_idx = sorted(idx_rank, key=lambda k: idx_rank[k], reverse=True)
        for idx in sorted_rank_idx[:int(len(sorted_rank_idx)*ratio)]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return ' '.join(summary)

<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/k5/_4q9447d6f727nxck_52wgy40000gn/T/ipykernel_46000/675051371.py:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [44]:
def summarise_contents(x):
    tr = TextRank() #window, coef, threshold
    token = tokenize(x)
    tr.loadSentence(token)

    tr.build(token)
    ranks = tr.get_rank()
    return tr.summarize(0.2)

In [45]:
with open("../data.json", 'r') as f:
    data = json.load(f)

In [46]:
contents = [data[1]["content"], data[10]["content"], data[15]["content"]]

In [47]:
content = data[1]["content"]

In [48]:
print(summarise_contents(content))

회동이 끝난 후 야당 원내대표들은 김태우 특검법 도입과 신재민 전 사무관의 폭로를 검증하기 위한 기획재정위원회 소집·청문회 등을 더불어민주당에 요청했지만, 홍영표 민주당 원내대표(인천 부평을)가 수용 불가 입장을 밝혔다고 전했다. 반면 홍영표 민주당 원내대표는 말을 아꼈다.


In [49]:
content = '''
이순신(李舜臣, 1545년 4월 28일 ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다. 본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)였으며, 한성 출신이었다. 문반 가문 출신으로 1576년(선조 9년) 무과(武科)에 급제[1]하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라좌도 수군절도사를 거쳐 정헌대부 삼도수군통제사에 이르렀다.

함경도 동구비보권관(董仇非堡權管), 1581년 발포 수군만호(鉢浦水軍萬戶)가 되었다가 전라좌수영의 오동나무를 베기를 거절한 일로 좌수사 성박의 미움을 받기도 했다. 이후 1584년 남병사의 군관과 건원보권관, 훈련원참군, 1586년 사복시주부를 거쳐 조산보만호 겸 녹도둔전사의(造山堡萬戶兼鹿島屯田事宜)로 부임했다. 조산만호 겸 녹둔도사의 재직 중 1587년(선조 20년) 9월의 여진족의 사전 기습공격으로 벌어진 녹둔도전투에서 이겼지만 피해가 커서, 북병사 이일의 탄핵을 받고 백의종군(白衣從軍)하는 위치에 서기도 했다. 그 뒤 두번째 여진족과의 교전에서 승전, 복직하였다. 그 뒤 전라관찰사 이광(李洸)에게 발탁되어 전라도 조방장, 선전관 등을 역임했다. 1589년 정읍현감 재직 중 유성룡의 추천으로 고사리첨사(高沙里僉使)가 되고, 절충장군(折衝將軍), 만포진첨사(滿浦鎭僉使), 진도군수 등을 거쳐 전라좌도수사가 되어 임진왜란을 만나게 되었다.

임진왜란 때 조선의 삼도수군통제사가 되어 부하들을 통솔하는 지도력, 뛰어난 지략, 그리고 탁월한 전략과 능수능란한 전술로 일본 수군과의 해전에서 연전연승하여 나라를 구한 성웅(聖雄)으로 추앙받고 있다. 노량 해전에서 전사한 뒤 선무공신 1등관에 추록되고 증 의정부우의정에 추증되고 덕풍군에 추봉되었다가, 광해군 때 다시 증 의정부좌의정에 추증되고 덕풍부원군에 추봉되었고, 정조 때에는 증 의정부영의정으로 가증(加贈)되었다.

고려 때 정5품 중랑장(中郎將)을 지낸 덕수 이씨의 시조 이돈수(李敦守)의 12대손이며, 조선 초 영중추부사(領中樞府事)를 지낸 이변(李邊)[2]의 후손이다. 외가는 초계 변씨(卞氏), 처가는 온양 방씨(方氏, 당시에는 상주 방씨)이다. 그의 묘는 충청남도 아산시에 있다
'''

In [50]:
print(summarise_contents(content))

1589년 정읍현감 재직 중 유성룡의 추천으로 고사리첨사(高沙里僉使)가 되고, 절충장군(折衝將軍), 만포진첨사(滿浦鎭僉使), 진도군수 등을 거쳐 전라좌도수사가 되어 임진왜란을 만나게 되었다. 고려 때 정5품 중랑장(中郎將)을 지낸 덕수 이씨의 시조 이돈수(李敦守)의 12대손이며, 조선 초 영중추부사(領中樞府事)를 지낸 이변(李邊)[2]의 후손이다.
